In [43]:
import pandas as pd
import numpy as np
import os
import torch
import os
import sys
sys.path.insert(0, os.path.abspath('../')) # needed to import src
from src.twitteraae.code.predict import Predict
from sklearn.model_selection import train_test_split
#print(os.getcwd())
# print(sys.path)
# import src.twitteraae.code.predict


In [44]:
gab_data = pd.read_csv('../data/GabHateCorpus_annotations.tsv', sep='\t', header=0)
gab_data

,ID,Annotator,Text,Hate,HD,CV,VO,REL,RAE,SXO,GEN,IDL,NAT,POL,MPH,EX,IM
0,27044,4,Ah the PSYOPS antifa crew is back. That’s how ...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,27044,15,Ah the PSYOPS antifa crew is back. That’s how ...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,27044,10,Ah the PSYOPS antifa crew is back. That’s how ...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,27044,8,Ah the PSYOPS antifa crew is back. That’s how ...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,27045,4,Get the new Android app update released today ...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86524,9188,6,"He thinks only peons own guns, you're supposed...",1,1,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
86525,9188,3,"He thinks only peons own guns, you're supposed...",0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86526,9222,11,America must stop funding the United Nations.,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86527,9222,3,America must stop funding the United Nations.,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
print('shape of GAB:', gab_data.shape) # (86529, 17)
print('# of unique posts:', len(gab_data['ID'].unique())) # 27665 

shape of GAB: (86529, 17)
# of unique posts: 27665


In [46]:
'''
processing of this dataset is done from section 4 of the paper here: https://osf.io/nqt6h/
CV ("Call for Violence"), HD (an "Assault on Human Dignity"), HD and CV, or NH ("Not Hateful").
If none apply, the document is to be considered NH ("Not Hateful"). Vulgarity/Offensive Language directed at an indi-
vidual (VO)

Targeted Populations
This column should be used to identify which types of groups are targeted by
the hateful rhetoric. Select all descriptors which apply.
- RAE: Race or ethnicity (includes anti-asian, anti-latino, anti-black, anti-arab, anti-semitism etc.)
- NAT: Nationality/regionalism (includes general xenophobia and targets against countries/regions)
- GEN: Gender (anti-woman, anti-man, anti-trans etc.)
- REL: Religion/spiritual beliefs (anti-muslim, anti-christian, etc.)
- SXO: Sexual Orientation
- IDL: Ideology (conservative/liberal/leftist/right-wing)
- POL: Political identification. Includes any reference to membership in a political organization (Democratic/Republican/ etc.).
- MPH: Mental/physical health status, physical disability.

Also EX/IM if the speech is explicit or implicit.

- turned into binary label with union of the cv/hd
'''


'\nprocessing of this dataset is done from section 4 of the paper here: https://osf.io/nqt6h/\nCV ("Call for Violence"), HD (an "Assault on Human Dignity"), HD and CV, or NH ("Not Hateful").\nIf none apply, the document is to be considered NH ("Not Hateful"). Vulgarity/Offensive Language directed at an indi-\nvidual (VO)\n\nTargeted Populations\nThis column should be used to identify which types of groups are targeted by\nthe hateful rhetoric. Select all descriptors which apply.\n- RAE: Race or ethnicity (includes anti-asian, anti-latino, anti-black, anti-arab, anti-semitism etc.)\n- NAT: Nationality/regionalism (includes general xenophobia and targets against countries/regions)\n- GEN: Gender (anti-woman, anti-man, anti-trans etc.)\n- REL: Religion/spiritual beliefs (anti-muslim, anti-christian, etc.)\n- SXO: Sexual Orientation\n- IDL: Ideology (conservative/liberal/leftist/right-wing)\n- POL: Political identification. Includes any reference to membership in a political organization (

In [47]:
# add binary label and investigation of aggregation methods
counts_df = gab_data.groupby(["ID"]).agg(num_annotators=('ID', 'count'),
                                         num_hate_label=('Hate','sum'),
                                         median_hate_label=('Hate','median'),
                                         mean_hate_label=('Hate','mean'))

# counts_df.rename(columns={'count':'num_annotators', 'sum':'num_hate_label','median':'median_hate_label', 'mean': 'mean_hate_label'}, inplace=True)
# counts_df.columns = counts_df.columns.droplevel(0)
counts_df['round_mean_hate_label'] = counts_df['mean_hate_label']
counts_df = counts_df.round({'round_mean_hate_label':0})

print('num of hate labels using median:', len(counts_df.loc[counts_df['median_hate_label']==1])) # 2337, in paper there is 2399
print('num of hate labels using mean rounded >0.5:', len(counts_df.loc[counts_df['round_mean_hate_label']==1])) # 2337, in paper there is 2399
print('num of hate labels using >= 1:', len(counts_df.loc[counts_df['num_hate_label']>=1])) #7813
print('num of hate labels using >1:', len(counts_df.loc[counts_df['num_hate_label']>1])) #2607
print('num_annotators summary stats:',counts_df['num_annotators'].describe())
# counts_df

num of hate labels using median: 2337
num of hate labels using mean rounded >0.5: 2337
num of hate labels using >= 1: 7813
num of hate labels using >1: 2607
num_annotators summary stats: count    27665.000000
mean         3.127743
std          0.393913
min          2.000000
25%          3.000000
50%          3.000000
75%          3.000000
max          8.000000
Name: num_annotators, dtype: float64


In [51]:
# join the label to each id
gab_data = gab_data.join(how='left',other=counts_df[['median_hate_label']], on='ID')
gab_data.fillna(0,inplace=True)
# gab_data.rename(columns={'median_hate_label':'is_hate'})
print(gab_data.columns.values)
# gab_data

['ID' 'Annotator' 'Text' 'Hate' 'HD' 'CV' 'VO' 'REL' 'RAE' 'SXO' 'GEN'
 'IDL' 'NAT' 'POL' 'MPH' 'EX' 'IM' 'median_hate_label']


In [52]:
# assuming here median label is majority
cleaned_gab_data = gab_data.groupby('ID',).agg(
    Text=('Text', 'first'),
    text_id=('ID','first'),
    is_hate=('median_hate_label', 'median'),
    hd=('HD', 'median'),
    cv=('CV', 'median'),
    vo=('VO', 'median'),
    rel=('REL', 'median'),
    rae=('RAE', 'median'),
    sxo=('SXO', 'median'),
    gen=('GEN', 'median'),
    idl=('IDL', 'median'),
    nat=('NAT', 'median'),
    pol=('POL', 'median'),
    mph=('MPH', 'median'),
    ex=('EX', 'median'),
    im=('IM', 'median'),
)
cleaned_gab_data

,Text,text_id,is_hate,hd,cv,vo,rel,rae,sxo,gen,idl,nat,pol,mph,ex,im
ID,,,,,,,,,,,,,,,,
0,Can't wait to see that Monday.,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,If you are from #Europe and care about #freeSp...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,#FREESAMHYDE http://www.thegatewaypundit.com/2...,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,most disgusting people: - third worlders - gay...,3,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,"JUST FOUND OUT BLACKS CONTROL THE GOVERNMENT, ...",4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27660,YESSSS. Impeach RR. Untenable. Intractably ...,27660,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27661,Not to mention presidents nixon reagan and bush,27661,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27662,Booker T Washington was an extraordinary Ameri...,27662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
twitteraae_predict = Predict(vocabfile='../src/twitteraae/model/model_vocab.txt',
                             modelfile='../src/twitteraae/model/model_count_table.txt')
twitteraae_predict.load_model()
# Output proportions are for the African-American, Hispanic, Asian, and White topics
#

C:\Users\matan\OneDrive\Documents\OMSCS\Thesis\Code\HateSpeech\notebooks


In [54]:
# recommended to assign dialict labels to tweets with dialect probabilities > 80%
cleaned_gab_data['twitter_aae'] = cleaned_gab_data.apply(lambda row: twitteraae_predict.predict(row['Text']), axis=1)
cleaned_gab_data['is_aae'] = cleaned_gab_data['twitter_aae'].str[0] >= 0.8
cleaned_gab_data['is_aae'] = cleaned_gab_data['is_aae'].astype(int) # only 1 is marked as aae by dialect
# cleaned_gab_data.loc[cleaned_gab_data['is_aae']==1]

In [40]:
# cleaned_gab_data.to_json(orient='records', lines=True, index=True, path_or_buf='../data/majority_gab_dataset_25k')
#todo get stratified from target population for 80/20 split.

In [64]:
# train_gab, test_gab = train_test_split(cleaned_gab_data, test_size=0.2, stratify=['is_hate'])

ValueError: Found input variables with inconsistent numbers of samples: [27665, 1]

In [107]:

stormfront = pd.read_csv('../data/stormfront/annotations_metadata.csv')
sf_content_dict = dict()
for file in stormfront['file_id']:
    with open(f'../data/stormfront/all_files/{file}.txt', "r", encoding='utf8') as sf_file:
        sf_content_dict[file] = sf_file.read()

sf_text_df = pd.DataFrame.from_dict(data=sf_content_dict, orient='index', columns=['text'])
sf_text_df.index.names = ['file_id']
stormfront = stormfront.join(how='left', other=sf_text_df, on='file_id')

In [115]:
stormfront['twitter_aae'] = stormfront.apply(lambda row: twitteraae_predict.predict(row['text']), axis=1)
stormfront['is_aae'] = stormfront['twitter_aae'].str[0] >= 0.8
stormfront['is_aae'] = stormfront['is_aae'].astype(int) # only 1 is marked as aae by dialect // 0 marked as aae

In [ ]:
stormfront.shape

(10944, 8)

In [117]:
stormfront

,file_id,user_id,subforum_id,num_contexts,label,text,twitter_aae,is_aae
0,12834217_1,572066,1346,0,noHate,"As of March 13th , 2014 , the booklet had been...","[0.03117249269491678, 0.017587739330060827, 0....",0
1,12834217_2,572066,1346,0,noHate,In order to help increase the booklets downloa...,"[0.0675233470132276, 0.024990787770922066, 0.8...",0
2,12834217_3,572066,1346,0,noHate,( Simply copy and paste the following text int...,"[0.07068404087345721, 0.036463286283906764, 0....",0
3,12834217_4,572066,1346,0,hate,Click below for a FREE download of a colorfull...,"[0.0878289733901142, 0.04044049034992705, 0.85...",0
4,12834217_5,572066,1346,0,noHate,Click on the `` DOWNLOAD ( 7.42 MB ) '' green ...,"[0.06770891657705622, 0.025514598021005915, 0....",0
...,...,...,...,...,...,...,...,...
10939,33676864_5,734541,1388,0,noHate,"Billy - `` That guy would n't leave me alone ,...","[0.07012772386434477, 0.025003537752268736, 0....",0
10940,33677019_1,735154,1388,0,noHate,Wish we at least had a Marine Le Pen to vote f...,"[0.06442548991852821, 0.04621656372273799, 0.8...",0
10941,33677019_2,735154,1388,0,noHate,Its like the choices are white genocide candid...,"[0.06341689439282158, 0.07569891312708113, 0.8...",0
10942,33677053_1,572266,1388,0,hate,Why White people used to say that sex was a si...,"[0.06961854541198273, 0.027863508063010634, 0....",0
